In [2]:
!pip install pandas openpyxl
!pip install nltk
!pip install textstat
!pip install vaderSentiment
!pip install better_profanity
!pip install joblib


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [14]:
# Import necessary libraries
import pandas as pd
import nltk
from better_profanity import profanity  # Using better_profanity for profanity check
import textstat
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import joblib

## Data annotations:

In [ ]:
# Check for profanity using the better_profanity library
def check_profanity(text):
    return profanity.contains_profanity(text)

def check_themes(text):
    sensitive_keywords = [
        "violence", "drugs", "sex", "explicit", "alcohol", "bomb", "gun", "fuck", 
        "abuse", "die", "kill", "unalive", "poison", "damn", "ghost", "scary", 
        "horror", "blood", "dead", "intimacy", "suggestive", "couple", "controversial", 
        "hot", "injuries", "romantic", "vampire", "shit", "sexy", "battle", "girlfriend", 
        "boyfriend", "scene kiss", "intercourse", "creepy", "naked", 
        "murder", "assault", "war", "massacre", "brutality", "stabbing", "shooting", 
        "gore", "torture", "hostility", "crime", "cocaine", "heroin", "nicotine", 
        "vaping", "addiction", "overdose", "substance abuse", "pills", "intoxicated", 
        "nudity", "erotic", "pornography", "fetish", "sensual", "lust", "strip", 
        "adult", "escort", "provocative", "cleavage", "lingerie", "scream", 
        "haunted", "supernatural", "zombie", "demon", "devil", "satanic", "cursed", 
        "exorcism", "nightmare", "fear", "terror", "trauma", "bullying", "harassment", 
        "suicide", "self-harm", "torture", "exploitation", "manipulation", 
        "oppression", "bastard", "hell", "slut", "whore", "damnation", "crap", 
        "douche", "asshole", "piss", "racism", "sexism", "discrimination", 
        "genocide", "terrorism", "dictatorship", "oppression", "xenophobia", 
        "cult", "stunt", "choking", "prank", "fire", "explosion", "theft", 
        "crime", "arrest", "kidnapping", "hostage", "escape", "affair", "adultery", 
        "infidelity", "lustful", "intimate", "sensual", "crush", "love triangle", 
        "gambling", "casino", "money laundering", "trafficking", "cheating", 
        "virus", "hacking", "dark web"
    ]
    text_lower = text.lower()
    return any(keyword in text_lower for keyword in sensitive_keywords)

# Check readability of the text using Flesch-Kincaid readability test
def check_readability(text):
    score = textstat.flesch_kincaid_grade(text)
    return score <= 7  # Grade level 5 is typically appropriate for kids

# Check sentiment tone using Vader sentiment analysis
def check_tone(text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_score = analyzer.polarity_scores(text)["compound"]
    return sentiment_score > -0.2  # Positive tone

# Check for cultural sensitivity by looking for offensive phrases
def check_cultural_sensitivity(text):
    offensive_phrases = ["racist", "stereotype", "offensive"]
    text_lower = text.lower()
    return any(phrase in text_lower for phrase in offensive_phrases)

# Analyze the transcript for all criteria
def analyze_transcript(text):
    analysis_results = {
        "profanity": check_profanity(text),
        "themes": check_themes(text),
        "readability": check_readability(text),
        "tone": check_tone(text),
        "cultural_sensitivity": check_cultural_sensitivity(text)
    }
    return analysis_results

# Concatenate relevant fields into a dictionary-like structure for analysis
def get_text_dict(row):
    return {
        "title": row.get("title", ""),
        "description": row.get("description", ""),
        "transcript": row.get("transcript", ""),
        "category_label": row.get("Category Label", ""),
        "channel_title": row.get("channel_title", ""),
    }

# Analyze if content is safe based on keywords in channel title or tags
def is_kids_safe(row):
    # Safely extract and lower-case the channel title and tags
    channel_title = str(row.get("channel_title", "")).lower()  # Convert to string if not already
    tags = str(row.get("tags", "")).lower()  # Convert to string if not already
    
    # Check if any safe keyword exists in the channel title or tags
    if any(keyword.lower() in channel_title for keyword in safe_channels):
        return True
    if any(keyword.lower() in tags for keyword in safe_tags):
        return True
    return False

safe_channels = [
        "kids", "Cartoon Network India", "SpongeBob SquarePants Official", "WB kids", 
        "Masha and The Bear", "Peppa Pig - Official Channel", "Shaun the Sheep Official", 
        "Disney XD", "CARTOON AKG", "cartoon", "Videogyan Shows - Educational Videos For Kids", 
        "Bluey - Official Channel", "Disney Jr.", "Disney", "Mr Bean World", "Cartoonito", 
        "ariki Anime", "anime", "Nick Jr", "nursery", "rhymes", "cocomelon", "Dodo Kids", "Chewy"
    ]
    
safe_tags = [
        "learning", "toddler", "kids", "babies", "preschool", "family friendly", 
        "anime", "cartoon", "animation", "art", "drawing", "cute", "children", 
        "tom and jerry", "spongebob", "Scooby-Doo!", "Tom and Jerry", "Looney Tunes", 
        "Bugs Bunny", "spider-man", "phineas and ferb", "baby"
    ]


In [49]:
# Load the datasets from Excel files
file_path = '../data/raw/combined_dataset.xlsx'  # Replace with your actual file path

df = pd.read_excel(file_path)

# Check the first few rows to verify successful loading
df.head()


,video_id,title,description,published_at,channel_title,tags,category_id,Category Label,default_audio_language,transcript,duration,view_count,like_count,dislike_count
0,R7KfBjoLGOY,Every Swear Word RANKED...,Every Swear Word RANKED... today we rank every...,2024-06-15T20:34:45Z,The Duck,NaN,1,Film & Animation,NaN,swear words a very essential part of the vocab...,NaN,NaN,NaN,NaN
1,GkQmkZNLCeM,Best Toy Learning Video for Toddlers and Kids ...,Best Toy Learning Video for Toddlers and Kids ...,2021-07-08T18:29:49Z,Genevieve's Playhouse - Learning Videos for Kids,"best, learning video for toddlers, for kids, c...",1,Film & Animation,en,Transcript not available,NaN,NaN,NaN,NaN
2,_et1i-ykdxA,Best ABC Learning Toy Video for Toddlers! Paw ...,Best ABC Learning Toy Video for Toddlers! Paw ...,2022-07-14T16:45:05Z,Genevieve's Playhouse - Learning Videos for Kids,"abc, for toddlers, for baby, learning, educati...",1,Film & Animation,en,Transcript not available,NaN,NaN,NaN,NaN
3,_UNe6tz28OI,Best Toy Learning Video for Toddlers and Kids ...,Best Toy Learning Video for Toddlers and Kids ...,2021-03-23T01:43:53Z,Genevieve's Playhouse - Learning Videos for Kids,"best, toddler learning videos, for kids, for b...",1,Film & Animation,en,Transcript not available,NaN,NaN,NaN,NaN
4,zamt_1PAwqE,The Joy of Sharing Short Episode | Fun Learni...,"Welcome to ""The Learning Show"", the exciting a...",2023-05-19T14:42:38Z,Videogyan Shows - Educational Videos For Kids,"shorts, learning for kids, educational videos,...",1,Film & Animation,en,Transcript not available,NaN,NaN,NaN,NaN


In [ ]:
# Function to apply analysis to each row, considering all fields and specific keyword checks
def apply_analysis(df):
    # Create a dictionary from text fields for analysis
    df["text_dict"] = df.apply(get_text_dict, axis=1)
    
    # Concatenate all text fields into a single string, handling missing values
    df["combined_text"] = df["text_dict"].apply(lambda d: " ".join(str(v) for v in d.values() if pd.notnull(v)))

    # Perform analysis on the combined text
    df["profanity"] = df["combined_text"].apply(check_profanity)
    df["themes"] = df["combined_text"].apply(check_themes)
    df["readability"] = df["combined_text"].apply(check_readability)
    df["tone"] = df["combined_text"].apply(check_tone)
    df["cultural_sensitivity"] = df["combined_text"].apply(check_cultural_sensitivity)
    df["is_kids_safe"] = df.apply(is_kids_safe, axis=1)

    return df

# Apply the analysis functions to the DataFrame
df_1 = apply_analysis(df)

# Check the columns after analysis
df_1[
    ["title", "transcript", "description", "Category Label", "channel_title", 
     "profanity", "themes", "readability", "tone", "cultural_sensitivity", "is_kids_safe"]
].head()


,title,transcript,description,Category Label,channel_title,profanity,themes,readability,tone,cultural_sensitivity,is_kids_safe
0,Every Swear Word RANKED...,swear words a very essential part of the vocab...,Every Swear Word RANKED... today we rank every...,Film & Animation,The Duck,True,True,False,True,False,False
1,Best Toy Learning Video for Toddlers and Kids ...,Transcript not available,Best Toy Learning Video for Toddlers and Kids ...,Film & Animation,Genevieve's Playhouse - Learning Videos for Kids,False,False,False,True,False,True
2,Best ABC Learning Toy Video for Toddlers! Paw ...,Transcript not available,Best ABC Learning Toy Video for Toddlers! Paw ...,Film & Animation,Genevieve's Playhouse - Learning Videos for Kids,False,False,False,True,False,True
3,Best Toy Learning Video for Toddlers and Kids ...,Transcript not available,Best Toy Learning Video for Toddlers and Kids ...,Film & Animation,Genevieve's Playhouse - Learning Videos for Kids,False,False,False,True,False,True
4,The Joy of Sharing Short Episode | Fun Learni...,Transcript not available,"Welcome to ""The Learning Show"", the exciting a...",Film & Animation,Videogyan Shows - Educational Videos For Kids,False,True,False,True,False,True


In [54]:
# Function to set the 'kids_safe' label based on analysis results
# A video is kids-safe if:
    # - No profanity
    # - No sensitive themes
    # - Readability is appropriate
    # - Positive tone
    # - No cultural sensitivity issues

def set_kids_safe(row):
    # Label as kids-safe (1) if all checks are passed, else label as not safe (0)
    if not row['profanity'] and not row['themes'] and row['tone'] and not row['cultural_sensitivity'] and row['is_kids_safe']:
        return 1  # Kids-safe
    return 0  # Not kids-safe

# Apply the function to set the 'kids_safe' label
df_1['kids_safe_nltk'] = df_1.apply(set_kids_safe, axis=1)

# Check the final results
df_1[['title', 'transcript', 'description', 'kids_safe_nltk']].head()

,title,transcript,description,kids_safe_nltk
0,Every Swear Word RANKED...,swear words a very essential part of the vocab...,Every Swear Word RANKED... today we rank every...,0
1,Best Toy Learning Video for Toddlers and Kids ...,Transcript not available,Best Toy Learning Video for Toddlers and Kids ...,1
2,Best ABC Learning Toy Video for Toddlers! Paw ...,Transcript not available,Best ABC Learning Toy Video for Toddlers! Paw ...,1
3,Best Toy Learning Video for Toddlers and Kids ...,Transcript not available,Best Toy Learning Video for Toddlers and Kids ...,1
4,The Joy of Sharing Short Episode | Fun Learni...,Transcript not available,"Welcome to ""The Learning Show"", the exciting a...",0


In [58]:
# Function to modify `kids_safe_nltk` for specific rows
def set_kids_safe_i(row):
    # Label as kids-safe (1) if all checks are passed, else label as not safe (0)
    if not row['profanity'] and not row['themes'] and row['tone'] and not row['cultural_sensitivity']:
        return 1  # Kids-safe
    return 0  # Not kids-safe

# Create a new DataFrame for the updated values
df_updated = df_1.copy()

# Apply the function to rows where `kids_safe_nltk` is 0 and create a new column
df_updated.loc[df_updated['kids_safe_nltk'] == 0, 'kids_safe_nltk'] = df_updated[df_updated['kids_safe_nltk'] == 0].apply(set_kids_safe_i, axis=1)

# Verify the updated DataFrame
df_updated.head()


,video_id,title,description,published_at,channel_title,tags,category_id,Category Label,default_audio_language,transcript,...,dislike_count,text_dict,combined_text,kids_safe_nltk,profanity,themes,readability,tone,cultural_sensitivity,is_kids_safe
0,R7KfBjoLGOY,Every Swear Word RANKED...,Every Swear Word RANKED... today we rank every...,2024-06-15T20:34:45Z,The Duck,NaN,1,Film & Animation,NaN,swear words a very essential part of the vocab...,...,NaN,"{'title': 'Every Swear Word RANKED...', 'descr...",Every Swear Word RANKED... Every Swear Word RA...,0,True,True,False,True,False,False
1,GkQmkZNLCeM,Best Toy Learning Video for Toddlers and Kids ...,Best Toy Learning Video for Toddlers and Kids ...,2021-07-08T18:29:49Z,Genevieve's Playhouse - Learning Videos for Kids,"best, learning video for toddlers, for kids, c...",1,Film & Animation,en,Transcript not available,...,NaN,{'title': 'Best Toy Learning Video for Toddler...,Best Toy Learning Video for Toddlers and Kids ...,1,False,False,False,True,False,True
2,_et1i-ykdxA,Best ABC Learning Toy Video for Toddlers! Paw ...,Best ABC Learning Toy Video for Toddlers! Paw ...,2022-07-14T16:45:05Z,Genevieve's Playhouse - Learning Videos for Kids,"abc, for toddlers, for baby, learning, educati...",1,Film & Animation,en,Transcript not available,...,NaN,{'title': 'Best ABC Learning Toy Video for Tod...,Best ABC Learning Toy Video for Toddlers! Paw ...,1,False,False,False,True,False,True
3,_UNe6tz28OI,Best Toy Learning Video for Toddlers and Kids ...,Best Toy Learning Video for Toddlers and Kids ...,2021-03-23T01:43:53Z,Genevieve's Playhouse - Learning Videos for Kids,"best, toddler learning videos, for kids, for b...",1,Film & Animation,en,Transcript not available,...,NaN,{'title': 'Best Toy Learning Video for Toddler...,Best Toy Learning Video for Toddlers and Kids ...,1,False,False,False,True,False,True
4,zamt_1PAwqE,The Joy of Sharing Short Episode | Fun Learni...,"Welcome to ""The Learning Show"", the exciting a...",2023-05-19T14:42:38Z,Videogyan Shows - Educational Videos For Kids,"shorts, learning for kids, educational videos,...",1,Film & Animation,en,Transcript not available,...,NaN,{'title': 'The Joy of Sharing Short Episode | ...,The Joy of Sharing Short Episode | Fun Learni...,0,False,True,False,True,False,True


In [44]:
df_1.head(10)

,video_id,title,description,published_at,channel_title,tags,category_id,Category Label,default_audio_language,transcript,...,dislike_count,text_dict,combined_text,profanity,themes,readability,tone,cultural_sensitivity,is_kids_safe,kids_safe_nltk
0,R7KfBjoLGOY,Every Swear Word RANKED...,Every Swear Word RANKED... today we rank every...,2024-06-15T20:34:45Z,The Duck,NaN,1,Film & Animation,NaN,swear words a very essential part of the vocab...,...,NaN,"{'title': 'Every Swear Word RANKED...', 'descr...",Every Swear Word RANKED... Every Swear Word RA...,True,True,False,True,False,False,0
1,GkQmkZNLCeM,Best Toy Learning Video for Toddlers and Kids ...,Best Toy Learning Video for Toddlers and Kids ...,2021-07-08T18:29:49Z,Genevieve's Playhouse - Learning Videos for Kids,"best, learning video for toddlers, for kids, c...",1,Film & Animation,en,Transcript not available,...,NaN,{'title': 'Best Toy Learning Video for Toddler...,Best Toy Learning Video for Toddlers and Kids ...,False,False,False,True,False,True,1
2,_et1i-ykdxA,Best ABC Learning Toy Video for Toddlers! Paw ...,Best ABC Learning Toy Video for Toddlers! Paw ...,2022-07-14T16:45:05Z,Genevieve's Playhouse - Learning Videos for Kids,"abc, for toddlers, for baby, learning, educati...",1,Film & Animation,en,Transcript not available,...,NaN,{'title': 'Best ABC Learning Toy Video for Tod...,Best ABC Learning Toy Video for Toddlers! Paw ...,False,False,False,True,False,True,1
3,_UNe6tz28OI,Best Toy Learning Video for Toddlers and Kids ...,Best Toy Learning Video for Toddlers and Kids ...,2021-03-23T01:43:53Z,Genevieve's Playhouse - Learning Videos for Kids,"best, toddler learning videos, for kids, for b...",1,Film & Animation,en,Transcript not available,...,NaN,{'title': 'Best Toy Learning Video for Toddler...,Best Toy Learning Video for Toddlers and Kids ...,False,False,False,True,False,True,1
4,zamt_1PAwqE,The Joy of Sharing Short Episode | Fun Learni...,"Welcome to ""The Learning Show"", the exciting a...",2023-05-19T14:42:38Z,Videogyan Shows - Educational Videos For Kids,"shorts, learning for kids, educational videos,...",1,Film & Animation,en,Transcript not available,...,NaN,{'title': 'The Joy of Sharing Short Episode | ...,The Joy of Sharing Short Episode | Fun Learni...,False,True,False,True,False,True,0
5,zlpfcKffgDA,DISGUISE | Short Horror Film,A woman encounters a malevolent presence in he...,2023-06-02T19:00:07Z,Tyler Czajkowski,"horror short film, horror short, short horror ...",1,Film & Animation,en-US,Transcript not available,...,NaN,"{'title': 'DISGUISE | Short Horror Film', 'des...",DISGUISE | Short Horror Film A woman encounter...,False,True,False,False,False,True,0
6,NZlJ6Saq-S4,Best Toy Learning Videos for Toddlers - Family...,Best Toy Learning Videos for Toddlers - Family...,2022-10-12T01:40:48Z,Genevieve's Playhouse - Learning Videos for Kids,"best, learning video for toddlers, for kids, c...",1,Film & Animation,en,Transcript not available,...,NaN,{'title': 'Best Toy Learning Videos for Toddle...,Best Toy Learning Videos for Toddlers - Family...,False,False,False,True,False,True,1
7,9_WBQISVHnw,Best Learning Video for Toddlers Learn Colors ...,Best Learning Video for Toddlers Learn Colors ...,2019-01-05T16:52:49Z,Genevieve's Playhouse - Learning Videos for Kids,"best, learning video for toddlers, for kids, c...",1,Film & Animation,en,Transcript not available,...,NaN,{'title': 'Best Learning Video for Toddlers Le...,Best Learning Video for Toddlers Learn Colors ...,False,False,False,True,False,True,1
8,y6Fo4i_pLyc,Toy Learning Video for Toddlers - Learn Spanis...,Toy Learning Video for Toddlers - Learn Spanis...,2020-04-23T18:12:57Z,Genevieve's Playhouse - Learning Videos for Kids,"best, toy learning, learning video, video for ...",1,Film & Animation,en,Transcript not available,...,NaN,{'title': 'Toy Learning Video for Toddlers - L...,Toy Learning Video for Toddlers - Learn Spanis...,False,False,False,True,False,True,1
9,WtSHDvX7Y9k,Creepy Playground! #Shorts #Sca

In [64]:
# Replace NaN values with empty strings
df_1 = df_1.fillna('')

In [33]:
df_1.columns

Index(['video_id', 'title', 'description', 'published_at', 'channel_title',
       'tags', 'category_id', 'Category Label', 'default_audio_language',
       'transcript', 'duration', 'view_count', 'like_count', 'dislike_count',
       'text_dict', 'combined_text', 'kids_safe_nltk', 'profanity', 'themes',
       'readability', 'tone', 'cultural_sensitivity', 'is_kids_safe'],
      dtype='object')

In [55]:
desired_order = ['video_id', 'title', 'description', 'published_at', 'channel_title',
       'tags', 'category_id', 'Category Label', 'default_audio_language',
       'transcript', 'duration', 'view_count', 'like_count', 'dislike_count',
       'text_dict', 'combined_text', 'kids_safe_nltk', 'profanity', 'themes', 'readability',
       'tone', 'cultural_sensitivity', 'is_kids_safe']

In [56]:
df_1 = df_1[desired_order]

In [59]:
df_updated = df_updated[desired_order]

In [57]:
# Save the labelled DataFrames to new Excel files
output_file_1 = '/Users/shusritavenugopal/Documents/MSIS/FALL2024/InformationStorageAndRetrieval/video_classification_project/data/raw/labelled_data_2.xlsx'  # Specify the desired output file name

# Use the variable output_file_1 to specify the file name
df_1.to_excel(output_file_1, engine='openpyxl', index=False)

print("Labelled data has been saved to Excel.")


Labelled data has been saved to Excel.


In [60]:
# Save the labelled DataFrames to new Excel files
output_file_1 = '/Users/shusritavenugopal/Documents/MSIS/FALL2024/InformationStorageAndRetrieval/video_classification_project/data/raw/labelled_data_3.xlsx'  # Specify the desired output file name

# Use the variable output_file_1 to specify the file name
df_updated.to_excel(output_file_1, engine='openpyxl', index=False)

print("Labelled data has been saved to Excel.")


Labelled data has been saved to Excel.
